## PDI - Rebalancing data for March 2023

## Configuring packages

In [15]:
import pandas as pd
import numpy as np
import requests
import decouple
import sys 
sys.path.append('../')

from web3 import Web3
from abis import index_anatomy, univ3_pool , erc20_contract


infura_url = decouple.config("ETHEREUM_INFURA_URL")
web3_object = Web3(Web3.HTTPProvider(infura_url))

In [16]:
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

In [17]:
from defillama import DefiLlama
llama = DefiLlama()

In [18]:
import plotly.express as px
import plotly.graph_objects as go

In [19]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Data retrieving and filtering

### Token inclusion criteria

1. **Project and token characteristics**
    1. The project must have a listing on Defi Llama.
    1. The project’s token should have been listed on CoinGecko with pricing data at least 1 year prior to the date of inclusion in the index. 
    1. The project should have a token that is native to Ethereum L1 or L2. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project. 
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. 
    1. Projects that have circular feedback characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.
   <br>
1. **Pricing requirmeents**
    1. Token should have a manipulation resistant price feed in the form of a Chainlink price feed or a Uniswap V3 TWAP oracle with a cardinality of at least 300 and a minimum liquidity level of $1mm.
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 30% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The token should have in aggregate at least \$5mm of onchain liquidity across Uniswap v2, Uniswap v3, Sushiswap, Balancer v1, Balancer v2, Bancor v2 and Bancor v3.
    1. The price should experience no more than 4% price impact when executing a $100k trade.
    1. The token must have shown consistent DeFi liquidity on Ethereum.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

### Tokens list

In [20]:
def normalize_symb(symb):
    return ''.join(c for c in symb if c.isalnum()).upper()

#### DefiLlama

In [21]:
protocols = llama.get_all_protocols()

In [22]:
top_defi_tokens = {} 
for pr in protocols[:1000]:
    if 'Ethereum' in pr['chains'] and pr['address'] and pr['address']!='-' and pr['symbol'] != '-':
        symbol = normalize_symb(pr['symbol']) 
        address = pr['address'].lower().split(":")[-1]
        top_defi_tokens[symbol] = {
            'symbol': symbol, 
            'address': address 
        }

# Switching NXM for WNXM

del top_defi_tokens['NXM']
top_defi_tokens['WNXM'] = {
    'symbol' : 'WNXM',
    'address' : '0x0d438f3b5175bebc262bf23753c1e53d03432bde'
}

#### CoinGecko

##### Tokens addresses

In [23]:
coins_list = pd.DataFrame(cg.get_coins_list(include_platform=True))

In [24]:
tokens = {} 
tokens_by_addr = {}
for index, coin in coins_list.iterrows():
    if len(coin['platforms']) >= 1 and 'ethereum' in list(coin['platforms'].keys()):
        symbol = normalize_symb(coin['symbol'])
        address = coin['platforms']['ethereum'].lower().split(":")[-1]
        token = {
            'symbol': symbol,
            'address': address 
        }
        tokens[symbol] = token
        tokens_by_addr[address] = token



In [25]:
# These token's addresses cant be found in the coingecko list of eth addresses
exclude_symbols = []
for symb, token in top_defi_tokens.items():
    if token['address'] not in tokens_by_addr.keys():
        exclude_symbols.append(token['symbol'])


In [26]:
for symb in exclude_symbols:
    del top_defi_tokens[symb]

top_defi_token_symbols = list(top_defi_tokens.keys())


##### Market data

In [27]:
cg_tokens_data = cg.get_coins_markets(vs_currency='USD', order='market_cap_desc')
for i in range(2, 20):
    cg_tokens_data.extend(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', page=i))
    
cg_tokens_data = pd.DataFrame.from_records(cg_tokens_data)
cg_tokens_data.set_index(cg_tokens_data['symbol'].str.upper(),inplace=True)

In [28]:
tokens_market_data = cg_tokens_data.set_index(cg_tokens_data['symbol'].str.upper())


In [29]:
duplicated = tokens_market_data.index[tokens_market_data.index.duplicated(keep='first')]
tokens_market_data = tokens_market_data.drop(index=duplicated)

In [30]:
tokens_market_data = tokens_market_data.rename(index={"ETH": "WETH"})
tokens_market_data.query('index in @top_defi_token_symbols',inplace=True)

### Token's supply check 

In [31]:

sp_passed_index = (tokens_market_data['circulating_supply'] / tokens_market_data['total_supply']) > 0.3
sp_passed = tokens_market_data[sp_passed_index]
        

### Retrieve historical data & livetime check

In [32]:
exclude_list = []

prices_data = pd.DataFrame()
marketcaps = pd.DataFrame() 
for index, data in sp_passed[['id', 'symbol']].iterrows():
    id_ = data['id']
    symbol = data['symbol'].upper()
    data = cg.get_coin_market_chart_by_id(id_, vs_currency='USD', days='max')
  
    df_prices = pd.DataFrame(data['prices'], columns=['date', symbol])
    df_prices = df_prices[df_prices[symbol] > 0]
    df_prices['date'] = pd.to_datetime(df_prices['date'], unit='ms').dt.date
    df_prices['date'] = pd.to_datetime(df_prices['date'])
    df_prices = df_prices.set_index('date', drop=True)
    df_prices = df_prices.loc[~df_prices.index.duplicated(keep='first')]
    
    if len(df_prices) < 365:
        print(f'Excluding {symbol}, prices data available only for {len(df_prices)} < 365 days')
        exclude_list.append(symbol)
        continue
    prices_data = pd.concat([prices_data, df_prices], axis=1)
    
    df_mcaps = pd.DataFrame(data['market_caps'], columns=['date', symbol])
    df_mcaps = df_mcaps[df_mcaps[symbol] > 0]
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'], unit='ms').dt.date
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'])
    df_mcaps = df_mcaps.set_index('date', drop=True)
    df_mcaps = df_mcaps.loc[~df_mcaps.index.duplicated(keep='first')]
    
    if len(df_mcaps) < 365:
        print(f'Note: {symbol}, marketcap data available only for {len(df_mcaps)} < 365 days')
        ## continue
    df_mcaps = df_mcaps.reindex(df_prices.index)
    marketcaps = pd.concat([marketcaps, df_mcaps], axis=1)
    

Note: FLOKI, marketcap data available only for 325 < 365 days
Excluding FRXETH, prices data available only for 50 < 365 days
Note: CFG, marketcap data available only for 141 < 365 days
Excluding EUL, prices data available only for 218 < 365 days
Excluding BTRFLY, prices data available only for 172 < 365 days
Excluding PSTAKE, prices data available only for 338 < 365 days
Note: DEUS, marketcap data available only for 335 < 365 days
Excluding BLUSD, prices data available only for 64 < 365 days
Excluding BRIDGE, prices data available only for 240 < 365 days
Excluding FLOOR, prices data available only for 333 < 365 days
Note: GAMMA, marketcap data available only for 353 < 365 days
Note: ORC, marketcap data available only for 355 < 365 days


### Non quantative checks

In [33]:
manual_exclusions = [
    {
        'symbol': 'WBTC',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'WETH',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'ERN',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'NFTX',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'RGT',
        'reason': 'Deprecated asset'
    },
    {
        'symbol': 'SUPER',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'BOND',
        'reason': 'Criteria 1.I'
    },
    {
        'symbol': 'USDC',
        'reason': 'stable'
    },
    {
        'symbol': 'FEI',
        'reason': 'stable'
    },
    {
        'symbol': 'SYN',
        'reason': 'bridge'
    },
    {
        'symbol': 'HBTC',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'BONE',
        'reason': 'Criteria 1.6'    
    },
    {
        'symbol': 'CRO',
        'reason': 'Not DeFi'
    }
    
    
]
for token in manual_exclusions:
    exclude_list.append(token['symbol'])

In [34]:
passed_q_and_non_q_checks = sp_passed.query('index not in @exclude_list' )


### Liquidity check

To derive a slippage threshold, consider an abstract univ2-like pool with $5m\$$ overall liquidity. 
I.e if pool has reserves $R_1$, $R_2$ and $p_1$ and $p_2$ are the correspondent tokens prices (in USD), then
$$R_1p_1 = R_2p_2 = 2.5m \$$$

Swapping $x$ token1 for token2 will result in the output amount $y$ such that
$$R_1\cdot R_2 = (R_1 + x) \cdot (R_2 - y)$$
$$y = \frac{R_2\cdot x}{R_1 + x}$$
Then the delivery price will be
$$p_d = \frac{y}{x} = \frac{R_2}{R_1 + x}$$
The spot price may be estimated as a delivery price for relatively small input amount $x_0$
$$p_s = \frac{y_0}{x_0} = \frac{R_2}{R_1 + x_0}$$

Then the theoretical slippage for this pool will be
$$slippage = \frac{p_d}{p_s} - 1 = \frac{R_1+x_0}{R_1+x} - 1 = \frac{p_1R_1+p_1x_0}{p_1R_1+p_1x}-1$$

In [35]:
liq = 5*10**6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5

In [36]:
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .5
max_slippage = round(2 * max_slippage, 2) / 2 
max_slippage

-0.04

In [37]:
url_0x = "https://api.0x.org/swap/v1/quote" 

In [38]:
decimals = 18
slippages = []
for symb, data in passed_q_and_non_q_checks.iterrows():
    try:
        query = {
            'buyToken': tokens[symb]['address'],
            'sellToken': 'ETH',
            'sellAmount': int(trade_value_tiny / float(cg_tokens_data[cg_tokens_data['symbol'] == 'eth']['current_price']) * 10 ** decimals),
            'enableSlippageProtection':'true'
        }
        
        # spot price is calculated as a price for 100$ swap
        resp = requests.get(url_0x, params=query)
        swap = resp.json()
        spot_price = float(swap['price'])
        
        query['sellAmount'] = int(trade_value / float(cg_tokens_data[cg_tokens_data['symbol'] == 'eth']['current_price']) * 10 ** decimals)
        resp = requests.get(url_0x, params=query)
        swap = resp.json()
        del_price = float(swap['price'])
        
        slippage = del_price / spot_price - 1
        slippages.append({
            'symb': symb,
            'spot_price': spot_price,
            'delivery_price': del_price,
            'slippage': slippage
        })
    except KeyError:
        continue
    

In [39]:
slippages_df = pd.DataFrame(slippages)
slippages_df.head(30)


,symb,spot_price,delivery_price,slippage
0,UNI,236.749220,235.445680,-0.005506
1,LDO,687.317549,684.543659,-0.004036
2,AAVE,18.133178,17.976831,-0.008622
3,SNX,627.415479,609.544176,-0.028484
4,RPL,40.700535,40.551778,-0.003655
5,FXS,156.772608,155.280004,-0.009521
6,CRV,1551.082202,1547.077407,-0.002582
7,MKR,2.413174,2.361606,-0.021369
8,1INCH,3031.933227,2935.306734,-0.031870
9,LRC,5161.969359,3779.949425,-0.267731


In [40]:
include_list = list(slippages_df[abs(slippages_df['slippage']) < abs(max_slippage)]['symb'])

liquidity_check = sp_passed.query('index in @include_list')

liquidity_check

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,6.660000,5017528583,20,6.656607e+09,1.226668e+08,6.790000,...,1.000000e+09,1.000000e+09,44.92,-85.19212,2021-05-03T05:25:04.822Z,1.030000,545.66093,2020-09-17T01:20:38.214Z,None,2023-01-27T14:10:22.125Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,2.280000,1911638559,36,2.285166e+09,1.105230e+08,2.450000,...,1.000000e+09,1.000000e+09,7.30,-68.70866,2021-08-20T08:35:20.158Z,0.406150,462.64133,2022-06-18T20:55:12.035Z,None,2023-01-27T14:10:48.450Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,87.040000,1239183231,42,1.392033e+09,1.229547e+08,90.240000,...,1.600000e+07,1.600000e+07,661.69,-86.85148,2021-05-18T21:19:59.514Z,26.020000,234.32719,2020-11-05T09:20:11.928Z,None,2023-01-27T14:10:27.853Z
SNX,havven,snx,Synthetix Network,https://assets.coingecko.com/coins/images/3406...,2.490000,778614556,58,7.804193e+08,4.152429e+07,2.520000,...,3.130405e+08,3.130405e+08,28.53,-91.26997,2021-02-14T01:12:38.505Z,0.034786,7060.48776,2019-01-06T00:00:00.000Z,"{'times': 3.988724071523694, 'currency': 'usd'...",2023-01-27T14:11:01.004Z
RPL,rocket-pool,rpl,Rocket Pool,https://assets.coingecko.com/coins/images/2090...,38.630000,738453743,61,7.384537e+08,9.970245e+06,39.600000,...,1.911341e+07,1.911341e+07,59.46,-34.95796,2021-11-09T04:03:31.003Z,0.008847,437002.95889,2018-08-28T00:00:00.000Z,None,2023-01-27T14:10:44.178Z
FXS,frax-share,fxs,Frax Share,https://assets.coingecko.com/coins/images/1342...,10.020000,735243141,62,1.001521e+09,2.000135e+07,10.750000,...,9.982298e+07,9.982298e+07,42.80,-76.55771,2022-01-12T15:22:27.465Z,1.500000,567.47688,2021-06-25T16:50:51.447Z,None,2023-01-27T14:10:59.428Z
CRV,curve-dao-token,crv,Curve DAO,https://assets.coingecko.com/coins/images/1212...,1.010000,731546322,63,3.335763e+09,8.250599e+07,1.069000,...,1.891867e+09,3.303030e+09,15.37,-93.43053,2020-08-14T00:00:00.000Z,0.331577,204.56412,2020-11-05T13:09:50.181Z,None,2023-01-27T14:10:36.437Z
MKR,maker,mkr,Maker,https://assets.coingecko.com/coins/images/1364...,650.390000,586171056,74,6.539809e+08,2.016121e+07,673.050000,...,9.776310e+05,1.005577e+06,6292.31,-89.64733,2021-05-03T21:54:29.333Z,168.360000,286.92814,2020-03-16T20:52:36.527Z,None,2023-01-27T14:11:01.565Z
1INCH,1inch,1inch,1inch,https://assets.coingecko.com/coins/images/1346...,0.518267,415343564,98,7.813897e+08,2.643642e+07,0.531876,...,1.500000e+09,1.500000e+09,8.65,-93.97886,2021-10-27T08:24:54.808Z,0.373690,39.40083,2022-12-30T12:35:04.113Z,None,2023-01-27T14:10:40.179Z


In [41]:
qualified_assets = liquidity_check.query('index not in @exclude_list')

qualified_assets

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,6.660000,5017528583,20,6.656607e+09,1.226668e+08,6.790000,...,1.000000e+09,1.000000e+09,44.92,-85.19212,2021-05-03T05:25:04.822Z,1.030000,545.66093,2020-09-17T01:20:38.214Z,None,2023-01-27T14:10:22.125Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,2.280000,1911638559,36,2.285166e+09,1.105230e+08,2.450000,...,1.000000e+09,1.000000e+09,7.30,-68.70866,2021-08-20T08:35:20.158Z,0.406150,462.64133,2022-06-18T20:55:12.035Z,None,2023-01-27T14:10:48.450Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,87.040000,1239183231,42,1.392033e+09,1.229547e+08,90.240000,...,1.600000e+07,1.600000e+07,661.69,-86.85148,2021-05-18T21:19:59.514Z,26.020000,234.32719,2020-11-05T09:20:11.928Z,None,2023-01-27T14:10:27.853Z
SNX,havven,snx,Synthetix Network,https://assets.coingecko.com/coins/images/3406...,2.490000,778614556,58,7.804193e+08,4.152429e+07,2.520000,...,3.130405e+08,3.130405e+08,28.53,-91.26997,2021-02-14T01:12:38.505Z,0.034786,7060.48776,2019-01-06T00:00:00.000Z,"{'times': 3.988724071523694, 'currency': 'usd'...",2023-01-27T14:11:01.004Z
RPL,rocket-pool,rpl,Rocket Pool,https://assets.coingecko.com/coins/images/2090...,38.630000,738453743,61,7.384537e+08,9.970245e+06,39.600000,...,1.911341e+07,1.911341e+07,59.46,-34.95796,2021-11-09T04:03:31.003Z,0.008847,437002.95889,2018-08-28T00:00:00.000Z,None,2023-01-27T14:10:44.178Z
FXS,frax-share,fxs,Frax Share,https://assets.coingecko.com/coins/images/1342...,10.020000,735243141,62,1.001521e+09,2.000135e+07,10.750000,...,9.982298e+07,9.982298e+07,42.80,-76.55771,2022-01-12T15:22:27.465Z,1.500000,567.47688,2021-06-25T16:50:51.447Z,None,2023-01-27T14:10:59.428Z
CRV,curve-dao-token,crv,Curve DAO,https://assets.coingecko.com/coins/images/1212...,1.010000,731546322,63,3.335763e+09,8.250599e+07,1.069000,...,1.891867e+09,3.303030e+09,15.37,-93.43053,2020-08-14T00:00:00.000Z,0.331577,204.56412,2020-11-05T13:09:50.181Z,None,2023-01-27T14:10:36.437Z
MKR,maker,mkr,Maker,https://assets.coingecko.com/coins/images/1364...,650.390000,586171056,74,6.539809e+08,2.016121e+07,673.050000,...,9.776310e+05,1.005577e+06,6292.31,-89.64733,2021-05-03T21:54:29.333Z,168.360000,286.92814,2020-03-16T20:52:36.527Z,None,2023-01-27T14:11:01.565Z
1INCH,1inch,1inch,1inch,https://assets.coingecko.com/coins/images/1346...,0.518267,415343564,98,7.813897e+08,2.643642e+07,0.531876,...,1.500000e+09,1.500000e+09,8.65,-93.97886,2021-10-27T08:24:54.808Z,0.373690,39.40083,2022-12-30T12:35:04.113Z,None,2023-01-27T14:10:40.179Z


### Marketcap ranking & filtering

In [42]:
chosen_tokens = qualified_assets.sort_values("market_cap", ascending=False)

Check for TWAP liquidity

In [59]:
# UNIv3 TWAP pools
current_twap_pools = [
    {
    'symbol': 'RBN',
    'pool address': '0xfe0df74636bc25c7f2400f22fe7dae32d39443d2'
    },
    {
    'symbol': "RPL",
    'pool address': '0xe42318eA3b998e8355a3Da364EB9D48eC725Eb45'
    }
]

def get_pool_liquidity(pool_dict):
    pool_contract = web3_object.eth.contract(address= web3_object.toChecksumAddress(pool['pool address']),abi= univ3_pool)
    token_0_address = pool_contract.functions.token0().call()
    token_1_address = pool_contract.functions.token1().call()
    token_0 = web3_object.eth.contract(address= token_0_address,abi=erc20_contract)
    token_1 = web3_object.eth.contract(address=token_1_address,abi = erc20_contract)
    token_0_balance = token_0.functions.balanceOf(web3_object.toChecksumAddress(pool['pool address'])).call() 
    token_1_balance = token_1.functions.balanceOf(web3_object.toChecksumAddress(pool['pool address'])).call()
    pool_liquidity = token_0_balance/(10**token_0.functions.decimals().call()) * float(cg.get_coin_info_from_contract_address_by_id('ethereum',token_0_address)['market_data']['current_price']['usd']) + token_1_balance/(10**token_1.functions.decimals().call()) * float(cg.get_coin_info_from_contract_address_by_id('ethereum',token_1_address)['market_data']['current_price']['usd'])
    return pool_liquidity

twap_warnings = []

for pool in current_twap_pools:
    if pool['symbol'] in chosen_tokens.index and get_pool_liquidity(pool) < 1000000:
        twap_warnings.append(pool['symbol'])

print(f"Please take note of the following under funded twap pools: {twap_warnings}")


Please take note of the following under funded twap pools: []


In [45]:
chosen_tokens_symbols = list(chosen_tokens.index)
prices_data = prices_data[chosen_tokens_symbols]
marketcaps = marketcaps[chosen_tokens_symbols]

In [46]:
without_nan_index = (marketcaps.isnull().sum(axis=1) == 0) & (prices_data.isnull().sum(axis=1) == 0)
marketcaps = marketcaps[without_nan_index]
prices_data = prices_data[without_nan_index]

## Weighting 

**Weighting requirements**
* The maximum weight any one token can have is 30%.
* All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
* Any asset with a weight below 1% will be removed from the index.


### Weight mcaps

In [47]:
weights = marketcaps.div(marketcaps.sum(axis=1), axis=0)
weights = weights.sort_values(weights.last_valid_index(), axis=1, ascending=False)

In [48]:
fig = px.line(weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    ## showlegend=False,
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights without max constraint'
)

### Adjust weights

This process adjusts weights to adhere to the weight constraints described in our methodology. In addition the `remove_tiny_weights` function checks for any assets with a weight below the minimum threshold of $0.5\%$ and proportionally increases all remaining weights (which are $> 0.5\%$). Note that after this procedure readjusting weights to respect maximum may be needed.

In [49]:
def adjust_weights(weights, max_weight):
    w = weights.copy()
    
    while (w > max_weight).any(axis=None):
        w[w > max_weight] = max_weight
        c = 1 - w.sum(axis=1)
        w_less = w[w < max_weight]
        w[(w < max_weight) & (w > 0)] += w_less.div(w_less.sum(axis=1), axis=0).mul(c, axis=0)
    
    return w

In [50]:
def remove_tiny_weights(weights, min_weight):
    w = weights.copy()
    
    w[w < min_weight] = 0
    w = w.div(w.sum(axis=1), axis=0)
    
    return w

In [51]:
adjusted_weights = adjust_weights(weights, max_weight=0.3) 
adjusted_weights = remove_tiny_weights(adjusted_weights, min_weight=0.01)
adjusted_weights = adjust_weights(adjusted_weights, max_weight=0.3)

adjusted_weights = adjusted_weights.sort_values(adjusted_weights.last_valid_index(),ascending=False, axis = 1)
# adjusted_weights

In [52]:
fig = px.line(adjusted_weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights with max constraint'
)

### Convert weights to the [1, 255] scale

On the contract side weights are integer numbers from the interval $[1, 255]$ with total sum $255$, so it's needed to convert retrieved weights to this format.

Note that one can't just round weights to integers after scaling, since it doesn't guarantee that their sum will be $255$. To fix that firstly floor function is applied to the weights and then $1$ is being added to the $k$ weights with largest fractional parts.

In [53]:
def convert_weights(weights):
    w_scaled = weights * 255
    w_res = np.floor(w_scaled).astype(int)    
    remainders = w_scaled - w_res    
    k = round(remainders.sum())
    w_below_max = w_res[w_res<76]
    for i in range(k):
        w_below_max[i] +=1
    for i in w_below_max.index:
        w_res[i] = w_below_max[i]
    return w_res

In [54]:
last_weights = adjusted_weights.iloc[-1].dropna()

In [55]:
converted_last_weights = convert_weights(last_weights)

## Summary table

In [56]:
pdi = pd.DataFrame()
pdi.index = chosen_tokens.index
pdi['name'] = chosen_tokens['name']
pdi['market_cap'] = marketcaps.iloc[-1].astype(int)
pdi['price'] = prices_data.iloc[-1]
pdi['weight'] = last_weights
pdi['weight_converted'] = converted_last_weights
pdi['address'] = [top_defi_tokens[symb]['address'] for symb in pdi.index]

pdi = pdi[pdi['weight'] > 0]
pdi = pdi.sort_values("market_cap",ascending=False)

In [57]:
#Querying current contract to ascertain which assets will be dropped

pdi_token_address = '0x632806BF5c8f062932Dd121244c9fbe7becb8B48'
ia_contract = web3_object.eth.contract(address=pdi_token_address,abi=index_anatomy)
addresses, weights = ia_contract.functions.anatomy().call()

for address in addresses:
    address = address.lower()
    if address not in list(pdi['address']):
        symbol = tokens_by_addr[address]['symbol']
        data = cg_tokens_data.loc[symbol]
        pdi.loc[symbol] = [data['name'],data['market_cap'],data['current_price'],0,0,address]
pdi

,name,market_cap,price,weight,weight_converted,address
symbol,,,,,,
UNI,Uniswap,4978519397,6.612763,0.300000,76,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984
LDO,Lido DAO,2104966609,2.523910,0.162355,42,0x5a98fcbea516cf06857215779fd812ca3bef1b32
AAVE,Aave,1225389859,86.143790,0.094514,25,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9
SNX,Synthetix Network,776453791,2.493497,0.059887,16,0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f
FXS,Frax Share,763880653,10.416069,0.058918,16,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0
RPL,Rocket Pool,760050404,40.190363,0.058622,15,0xd33526068d116ce69f19a9ee46f0bd304f21a51f
CRV,Curve DAO,747565477,1.033971,0.057659,15,0xd533a949740bb3306d119cc777fa900ba034cd52
MKR,Maker,602254935,667.968893,0.046452,12,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2
1INCH,1inch,428814831,0.536892,0.033074,9,0x111111111117dc0aa78b770fa6a738034120c302


In [58]:
zipped_assets = list(zip(list(pdi['address']),list(pdi['weight_converted'])))
sorted_assets = sorted(zipped_assets,key = lambda x: int(x[0],base=0) )

asset_string = ''
for i in sorted_assets:
    asset_string += f'{i[0]},'
asset_string = asset_string[:-1]
print(asset_string)
print([x[1] for x in sorted_assets])

0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e,0x111111111117dc0aa78b770fa6a738034120c302,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b,0x5a98fcbea516cf06857215779fd812ca3bef1b32,0x6123b0049f904d730db3c36a31167d9d4121fa6b,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,0xba100000625a3754423978a60c9317c58a424e3d,0xc00e94cb662c3520282e6f5717214004a7f26888,0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f,0xd33526068d116ce69f19a9ee46f0bd304f21a51f,0xd533a949740bb3306d119cc777fa900ba034cd52
[4, 9, 76, 16, 7, 42, 3, 4, 25, 12, 5, 6, 16, 15, 15]
